In [1]:
import numpy as np
from pyfinite import ffield

In [2]:
#Final Matrices as obtained from Cryptanalysis.ipynb
At = [[84, 112, 13, 102, 111, 25, 8, 67], [0, 70, 30, 16, 62, 50, 123, 3], [0, 0, 43, 3, 0, 25, 23, 74], [0, 0, 0, 12, 104, 51, 103, 26], [0, 0, 0, 0, 112, 99, 25, 10], [0, 0, 0, 0, 0, 11, 88, 66], [0, 0, 0, 0, 0, 0, 27, 30], [0, 0, 0, 0, 0, 0, 0, 38]]
E = [22, 115, 38, 69, 92, 43, 20, 26]
A = [[At[j][i] for j in range(len(At))] for i in range(len(At[0]))]
A

[[84, 0, 0, 0, 0, 0, 0, 0],
 [112, 70, 0, 0, 0, 0, 0, 0],
 [13, 30, 43, 0, 0, 0, 0, 0],
 [102, 16, 3, 12, 0, 0, 0, 0],
 [111, 62, 0, 104, 112, 0, 0, 0],
 [25, 50, 25, 51, 99, 11, 0, 0],
 [8, 123, 23, 103, 25, 88, 27, 0],
 [67, 3, 74, 26, 10, 66, 30, 38]]

In [3]:
block_size = 8
F = ffield.FField(7, gen=0x83, useLUT=-1)
A = np.array((A))
augmentedA = np.zeros((block_size, block_size*2), dtype = int)
inverseA = np.zeros((block_size, block_size), dtype = int)
Einverse = np.zeros((128, 128), dtype = int)

exponents = []
for i in range(0,128):
    exponents.append([1])

for base in range(0,128):
    for exponent in range(1,127):
        temp = exponents[base][exponent-1]
        result = F.Multiply(temp, base)
        exponents[base]+=[result]

for base in range(0,128):
    for exponent in range(1,127):
        Einverse[exponent][exponents[base][exponent]] = base

inverses = [1]
for i in range(1, 128):
    inverses+=[F.Inverse(i)]
    assert F.Multiply(i, inverses[i]) == 1


for i in range(0,block_size):
    for j in range(0,block_size):
        augmentedA[i][j] = A[i][j]
    augmentedA[i][i+j+1] = 1

for j in range(0,block_size):
    assert np.any(augmentedA[j:,j] != 0) # assert pivot row exists: A is invertible
    pivot_row = np.where(augmentedA[j:,j] != 0)[0][0] + j
    augmentedA[[j, pivot_row]] = augmentedA[[pivot_row, j]]
    # make pivot k 1
    mul_fact = inverses[augmentedA[j][j]]
    for k in range(block_size*2):
        augmentedA[j][k] = F.Multiply(augmentedA[j][k], mul_fact)
    for i in range(0,block_size):
        if i!=j and augmentedA[i][j] != 0:
            mult_fact = augmentedA[i][j]
            for k in range(block_size*2):
                temp = F.Multiply(augmentedA[j][k], mult_fact)
                augmentedA[i][k] = F.Add(temp, augmentedA[i][k])

for i in range(0,block_size):
    for j in range(0,block_size):
        inverseA[i][j] = augmentedA[i][block_size+j]
print("A inverse matrix: \n{}".format(inverseA))

A inverse matrix: 
[[ 67   0   0   0   0   0   0   0]
 [ 81  77   0   0   0   0   0   0]
 [ 26   5  96   0   0   0   0   0]
 [ 13  15  24  94   0   0   0   0]
 [ 80   4  41 120  19   0   0   0]
 [ 27  99  25  92  23  74   0   0]
 [ 55  83  71   5  17 112  14   0]
 [ 81  30 123  87  64 127  48  56]]


In [7]:
password = "gsfojqmrimffismjfkjtkpkujlmjhjkp" #Encrypted password
block_size = 16
num_blocks = int(len(password) / block_size) # 2 blocks

def Einverse_calculator(block, E):
    transformed = []
    for j in range(0,8):
        transformed+=[Einverse[E[j]][block[j]]]
    return transformed

def LinearTransform(msg,linmat):
    ans = [0]*8
    for i in range(8):
        temp  = []
        mul = [F.Multiply(linmat[i][j],msg[i]) for j in range(8)]
        for k in range(8):
            temp.append(np.bitwise_xor(ans[k],mul[k]))
        ans = temp
    return ans
def Ainverse_calculator(block, A):
    transformed = []
    for row_num in range(0,8):
        elem_sum = 0
        for col_num in range(0,8):
            elem = F.Multiply(A[row_num][col_num], block[col_num])
            elem_sum = F.Add(elem, elem_sum)
        transformed+=[elem_sum]
    return transformed

decrypted_password = ""
for i in range(0,2): # Decipher both the blocks of the password
    elements = password[block_size*i:block_size*(i+1)]
    currentBlock = []
    for j in range(0,8):
        currentBlock+=[(ord(elements[2*j]) - ord('f'))*16 + (ord(elements[2*j+1]) - ord('f'))]
    
    EA = Ainverse_calculator(Einverse_calculator(currentBlock, E), inverseA)
    EAE = Einverse_calculator(EA, E)
    
    EAEA = Ainverse_calculator(EAE, inverseA)
    EAEAE = Einverse_calculator(EAEA, E)
    
    print("Inverse EAEAE values for block:",str(i)+": ",EAEAE)
    for ch in EAEAE:
        decrypted_password += chr(ch)
print("The password is",decrypted_password)
    

TypeError: only integer scalar arrays can be converted to a scalar index